In [1]:
import pandas as pd
import matplotlib as plt
import seaborn as sns
import numpy as np

In [2]:
df=pd.read_csv('trainset.csv')


In [3]:
validation=pd.read_csv('testset.csv')

In [4]:
for col in df.columns:
    #print(df[col].unique())
    if df[col].isna().sum()>500:
        df=df.drop(col,axis=1)

In [5]:
df

,cls,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,VE1_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,1,0,0,25.001104,2.501652,4.923679,25.001104,1.250055,3.907976,3.763026,...,9.918524,67.526731,284.075684,8.877365,764,32,100,119,7.777778,4.666667
1,1,0,0,20.274271,2.409184,4.779725,20.274271,1.267142,3.699349,3.675755,...,9.636588,61.986524,226.038985,9.827782,429,23,82,96,6.166667,3.583333
2,1,0,0,23.679070,2.528665,4.931112,23.679070,1.392886,3.823524,3.787841,...,10.022647,65.026165,223.074562,8.579791,461,29,98,121,3.416667,3.638889
3,1,0,0,27.774585,2.557762,4.967131,27.774585,1.388729,4.000936,4.113451,...,10.250228,70.773493,266.069142,8.868971,735,35,120,150,3.888889,4.166667
4,1,0,0,39.613943,2.477563,4.835703,39.613943,1.320465,4.331178,4.133872,...,10.233043,79.537169,415.183067,7.548783,2674,45,156,182,8.500000,6.694444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,0,0,1,25.162228,2.444418,4.888835,25.162228,1.324328,3.885650,3.788230,...,9.911108,52.566001,259.120843,7.197801,688,30,100,118,5.416667,4.194444
7996,0,0,0,25.266178,2.534987,4.934272,25.266178,1.203151,3.960779,3.746040,...,10.024155,69.315801,328.076075,8.633581,904,34,108,128,9.250000,4.611111
7997,0,2,0,23.921252,2.296174,4.541085,23.921252,1.259013,3.850677,3.853332,...,9.488048,65.073826,278.094980,8.427121,813,23,92,103,6.305556,4.361111
7998,0,0,0,35.341681,2.541818,5.014881,35.341681,1.359295,4.226118,4.327006,...,10.356186,75.559148,352.169939,7.492977,1547,44,146,177,6.000000,5.583333


In [36]:
for row in df.index:
    if df.loc[row].isna().sum()>0:
        df.drop(row)

In [37]:
df.

11424000

In [6]:
import torch
import torch.nn as nn

In [7]:
y=df.cls.values
#x=df.drop('cls',axis=1).drop('ABC',axis=1).drop('ABCGG',axis=1).drop('n5FAHRing',axis=1).values
x=df.drop('cls',axis=1).values

In [8]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=69)

In [9]:
from torch.utils.data import Dataset, DataLoader
class dtset(Dataset):
    
    # Init
    def __init__(self,data,labels):
        self.data=torch.FloatTensor(data)
        self.labels=torch.FloatTensor(labels)
    
    # len
    def __len__(self):
        return len(self.data)
    
    # obtain item
    def __getitem__(self,idx):
        return self.data[idx],self.labels[idx]

In [10]:
# Autoencoder to reduce dimensionality
class Autoencoder(nn.Module):
    
    def __init__(self, input_size, encoding_dim):
        
        super(Autoencoder, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, encoding_dim)
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, input_size)
        )

    def forward(self, x):
        
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        
        return encoded, decoded

Request info on why ABC, ABCGG,nAcid,nBase are 0

In [11]:
class cnn(nn.Module):
    
    def __init__(self,input_size,num_classes=2,drop_per=0.03):
        
        super(cnn,self).__init__()
        
        # 1st convolution
        self.conv1d1 = nn.Conv1d(in_channels=input_size,out_channels=200,kernel_size=3)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool1d(kernel_size=3)
        self.dropout1 = nn.Dropout(p=drop_per)
        
        # 2nd convolution 
        self.conv1d2 = nn.Conv1d(in_channels=200,out_channels=150,kernel_size=3)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool1d(kernel_size=3)
        self.dropout2 = nn.Dropout(p=drop_per)
        
        # 3rd convolution
        self.conv1d3 = nn.Conv1d(in_channels=150,out_channels=100,kernel_size=3)
        self.relu3 = nn.ReLU()
        self.maxpool3 = nn.MaxPool1d(kernel_size=3)
        self.dropout3 = nn.Dropout(p=drop_per)
        
        # 4th convolution
        self.conv1d4 = nn.Conv1d(in_channels=100,out_channels=50,kernel_size=3)
        self.relu4 = nn.ReLU()
        self.maxpool4 = nn.MaxPool1d(kernel_size=3)
        self.dropout4 = nn.Dropout(p=drop_per)
        
        # 5th convolution
        self.conv1d5 = nn.Conv1d(in_channels=50,out_channels=1,kernel_size=3)
        self.sigmoid1 = nn.Sigmoid()
    
    def forward(self, x):

        # 1st convolution
        x = self.conv1d1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)
        x = self.dropout1(x)

        # 2nd convolution
        x = self.conv1d2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)
        x = self.dropout2(x)

        # 3rd convolution
        x = self.conv1d3(x)
        x = self.relu3(x)
        x = self.maxpool3(x)
        x = self.dropout3(x)

        # 4th convolution
        x = self.conv1d4(x)
        x = self.relu4(x)
        x = self.maxpool4(x)
        x = self.dropout4(x)

        # 5th convolution
        x = self.conv1d5(x)
        x = self.sigmoid1(x)

        # Flatten the output
        x = x.view(x.size(0), -1)

        return x


In [12]:
size=x.shape[1]

In [13]:
size

1427

In [14]:
batch_size=16

# create dataset objects
train=dtset(x_train,y_train)
test=dtset(x_test,y_test)

# create loaders
train_loader=DataLoader(train,batch_size=batch_size,shuffle=True)
test_loader=DataLoader(test,batch_size=batch_size)

In [15]:
x_train.shape,y_train.shape

((6400, 1427), (6400,))

In [16]:
data=pd.DataFrame([i for i in train_loader])


In [17]:
x_test.shape,y_test.shape

((1600, 1427), (1600,))

In [18]:
import torch.optim as optim

encoding_dims=240
# Init autoencoder
ae=Autoencoder(input_size=size,encoding_dim=encoding_dims)

# Init autoencoder loss func MSE
ae_loss_func=nn.MSELoss()

# Init autencoder optimzer adam
ae_optimizer=optim.Adam(ae.parameters(),lr=0.0001 , eps=1e-6)

In [19]:
# Init model
model=cnn(input_size=encoding_dims, num_classes=2, drop_per=0.03)

# Init loss func crossentropy
loss_func=nn.CrossEntropyLoss()

# Init optimizer adam
optimizer=optim.Adam(model.parameters(),lr=0.001, eps = 1e-6)

In [20]:
ae_epochs=5

# Run for epochs
for epoch in range(ae_epochs):
    # Run for items in loader
    for data in train_loader:
        inputs,_=data # ignore labels
        ae_optimizer.zero_grad()
        encoded,decoded=ae(inputs)
        ae_loss=ae_loss_func(decoded,inputs)
        ae_loss.backward()
        ae_optimizer.step()
    print(f'Epoch [{epoch + 1}/{ae_epochs}], Loss: {ae_loss.item():.4f}')

/mnt/stor/miniconda/envs/torch/lib/python3.11/site-packages/torch/autograd/__init__.py:200: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343995622/work/c10/cuda/CUDAFunctions.cpp:109.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch [1/5], Loss: nan
Epoch [2/5], Loss: nan
Epoch [3/5], Loss: nan
Epoch [4/5], Loss: nan
Epoch [5/5], Loss: nan


In [21]:

epochs=10

# Run for epochs
for epoch in range(epochs):
    # Run for items in loader
    for inputs,labels in train_loader:
        optimizer.zero_grad()
        outputs=model(inputs)
        loss=loss_func(outputs,labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')
       
# Save
torch.save(ae.state_dict(),'models/autoencoder.pth')
torch.save(model.state_dict(),'models/model.pth')

# Load
ae.load_state_dict(torch.load('models/autoencoder.pth'))
model.load_state_dict(torch.load('models/model.pth'))

RuntimeError: Given groups=1, weight of size [200, 240, 3], expected input[1, 16, 1427] to have 240 channels, but got 16 channels instead